In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import pickle
from nltk.corpus import stopwords
import time
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import nltk
import json
#nltk.download('all')
print('TensorFlow Version: {}'.format(tf.__version__))

#현재 1.1에 맞게 바꾸어 놓았음. 1월 2일 4:33/

TensorFlow Version: 1.2.1


## Insepcting the Data

In [3]:
reviews = pd.read_csv("title_cont_sum_sim_norm_part.csv", encoding="cp949")

In [4]:
reviews.shape

(17184, 5)

In [5]:
reviews.head()

,article_title,article_cont,summarization,sum_type,sim
0,삼성 합병 도와주라 지시한 적 없 다 핵심 의혹 전면 부인,박근혜 대통령 일 오후 청와대 상춘재 출입 간담회 있 청와대 제공 박근혜 대통령...,청와대 제공 박근혜 대통령 일 오후 청와대 출입 만난 자리 작정 그 검찰 특검 언론...,textrank,123.547516
1,년 신년 일성 은 국민 들께 미안 달라진 朴 위상,권한정지 아니 년 신년사 통해 국정 자신감 피력탄핵 의결 대통령 권한 정지 박근혜 ...,권한정지 아니 년 신년사 통해 국정 자신감 피력 의결 대통령 권한 정지 박근혜 대통...,lexrank,101.711639
2,고초 겪어 마음 아파 朴 측근 재벌 에 동정론 제기,일 예정 없 긴급 간담회 연 박근혜 대통령 최순실 사태 연루 사법처리 측근 수사 받...,박 대통령 기업인들 생각 거기 미안한 마음 많 정부 시책 공감해 그분들 동참해 준 ...,textrank,103.974892
3,朴 대통령 각종 의혹 전면 부인 헌재 특검 대비 여론전,전신 박근혜 대통령 일 청와대 상춘재 출입단 신년 인사 겸한 티타임 있 전신 박근...,상황 박 대통령 헌재 심판 특검 수사 적극적인 방어 논리 펼친 것 여론 향배 어느 ...,lexrank,116.510712
4,朴 대통령 전격 의혹 전면 부인 헌재심판 여론전 본격 대응,박근혜 대통령 정유년 새해 첫날인 일 오후 청와대 상춘재 출입단 신년 인사회 겸한 ...,박 대통령 자리 세월호 참사 당일 미용 시술 의혹 삼성물산 제일모직 합병 지원 지시...,lexrank,146.351715


In [6]:
reviews.rename(columns={reviews.columns[2]:"text"},inplace=True)
reviews.rename(columns={reviews.columns[0]:"summary"},inplace=True)

In [7]:
# Check for any nulls values
reviews.isnull().sum() 

summary         0
article_cont    0
text            0
sum_type        0
sim             0
dtype: int64

In [8]:
# Inspecting some of the reviews
for i in range(5):
    print("Review #",i+1)
    print(reviews.summary[i])
    print(reviews.text[i])
    print()

Review # 1
삼성 합병 도와주라 지시한 적 없 다 핵심 의혹 전면 부인
청와대 제공 박근혜 대통령 일 오후 청와대 출입 만난 자리 작정 그 검찰 특검 언론 등 제기 의혹 전면 부인 박 대통령 청와대 블랙리스트 작성 깊숙 관여 의혹 대해 전혀 모르 일 전혀 알지 못 일 강 부인

Review # 2
년 신년 일성 은 국민 들께 미안 달라진 朴 위상
권한정지 아니 년 신년사 통해 국정 자신감 피력 의결 대통령 권한 정지 박근혜 대통령 일 취임 년차 신년 일성 국민 미안 언급 내놨 박 대통령 전년 월일 다음 해 신년사 발표해왔 권한 행사 정지 년 신년사 내지 못

Review # 3
고초 겪어 마음 아파 朴 측근 재벌 에 동정론 제기
박 대통령 기업인들 생각 거기 미안한 마음 많 정부 시책 공감해 그분들 동참해 준 것 압수수색 받고 여러 가지 어려움 많 겪 보 정말 그 제 굉장히 미안 그래서 마음 편 날 없 말

Review # 4
朴 대통령 각종 의혹 전면 부인 헌재 특검 대비 여론전
상황 박 대통령 헌재 심판 특검 수사 적극적인 방어 논리 펼친 것 여론 향배 어느 영향 줄 수 있 박 대통령 우선 주요 탄핵 사유 중 하나 특검 정조준 있 사안인 세월호 시 행적 설명 공 세간 미용시술 등 의혹 강 반박

Review # 5
朴 대통령 전격 의혹 전면 부인 헌재심판 여론전 본격 대응
박 대통령 자리 세월호 참사 당일 미용 시술 의혹 삼성물산 제일모직 합병 지원 지시 의혹 정유라 친구 아버지 운영 코퍼레이션 납품 강요 의혹 부인 헌재 심판 결정적 잣대 최순실과 공모 의혹 뇌물죄 의혹 시술 설 세월호 참사 당일 의혹 망라 부인한 것



In [9]:
reviews.head()

,summary,article_cont,text,sum_type,sim
0,삼성 합병 도와주라 지시한 적 없 다 핵심 의혹 전면 부인,박근혜 대통령 일 오후 청와대 상춘재 출입 간담회 있 청와대 제공 박근혜 대통령...,청와대 제공 박근혜 대통령 일 오후 청와대 출입 만난 자리 작정 그 검찰 특검 언론...,textrank,123.547516
1,년 신년 일성 은 국민 들께 미안 달라진 朴 위상,권한정지 아니 년 신년사 통해 국정 자신감 피력탄핵 의결 대통령 권한 정지 박근혜 ...,권한정지 아니 년 신년사 통해 국정 자신감 피력 의결 대통령 권한 정지 박근혜 대통...,lexrank,101.711639
2,고초 겪어 마음 아파 朴 측근 재벌 에 동정론 제기,일 예정 없 긴급 간담회 연 박근혜 대통령 최순실 사태 연루 사법처리 측근 수사 받...,박 대통령 기업인들 생각 거기 미안한 마음 많 정부 시책 공감해 그분들 동참해 준 ...,textrank,103.974892
3,朴 대통령 각종 의혹 전면 부인 헌재 특검 대비 여론전,전신 박근혜 대통령 일 청와대 상춘재 출입단 신년 인사 겸한 티타임 있 전신 박근...,상황 박 대통령 헌재 심판 특검 수사 적극적인 방어 논리 펼친 것 여론 향배 어느 ...,lexrank,116.510712
4,朴 대통령 전격 의혹 전면 부인 헌재심판 여론전 본격 대응,박근혜 대통령 정유년 새해 첫날인 일 오후 청와대 상춘재 출입단 신년 인사회 겸한 ...,박 대통령 자리 세월호 참사 당일 미용 시술 의혹 삼성물산 제일모직 합병 지원 지시...,lexrank,146.351715


## Preparing the Data

In [10]:
clean_texts = reviews.text
clean_summaries = reviews.summary

In [11]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1 # 단어 갯수를 왜 세지 ??????

In [12]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts)) # 단어 갯수를 왜 세지 ??????

Size of Vocabulary: 34528


In [14]:
f = open('w2v_title_sum_part.txt','rb')
embeddings_index = pickle.load(f)

print('Word embeddings:', len(embeddings_index))

Word embeddings: 16899


In [15]:
word_counts.items()

dict_items([('삼성', 424), ('합병', 126), ('도와주라', 5), ('지시한', 103), ('적', 233), ('없', 1693), ('다', 1273), ('핵심', 420), ('의혹', 1144), ('전면', 130), ('부인', 755), ('년', 1840), ('신년', 94), ('일성', 5), ('은', 1282), ('국민', 1566), ('들께', 6), ('미안', 8), ('달라진', 19), ('朴', 1001), ('위상', 48), ('고초', 5), ('겪어', 5), ('마음', 149), ('아파', 10), ('측근', 40), ('재벌', 50), ('에', 2850), ('동정론', 2), ('제기', 248), ('대통령', 24495), ('각종', 112), ('헌재', 851), ('특검', 2149), ('대비', 139), ('여론전', 67), ('전격', 77), ('헌재심판', 2), ('본격', 123), ('대응', 711), ('박근혜', 2605), ('정부', 3441), ('최순실', 1394), ('예산', 172), ('내역', 16), ('조사단서', 1), ('수집', 21), ('차원', 270), ('부정청탁', 6), ('수사', 1230), ('본격화', 63), ('이번', 707), ('주', 699), ('수뇌부', 35), ('줄소환', 4), ('일만', 58), ('침묵', 66), ('깬', 8), ('심경', 13), ('토', 26), ('로', 974), ('작심', 11), ('새해', 103), ('첫날', 63), ('깜짝', 107), ('기자', 50), ('간담회', 558), ('무엇', 31), ('을', 316), ('노렸나', 3), ('나', 589), ('를', 116), ('완전히', 75), ('엮', 24), ('것', 1875), ('뇌물죄', 76), ('모두', 398), ('세월호', 612), 

In [23]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold =2

for word, count in word_counts.items(): # word_count 내에 단어들 순서는 항상 같다. 
    if count >= threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100 # word_embedings에 없는 단어 비율 
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 5998
Percent of words that are missing from vocabulary: 17.37%


I use a threshold of 20, so that words not in CN can be added to our word_embedding_matrix, but they need to be common enough in the reviews so that the model can understand their meaning.

In [24]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts)) # 쓰인 단어 중복제외하고 쓰인 갯수
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 34528
Number of words we will use: 22901
Percent of words we will use: 66.33%


In [27]:
vocab_to_int['삼성']

0

In [28]:
int_to_vocab[1]

'합병'

In [29]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word] # [i] 하면 각 행을 의미함 
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

22901


In [30]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word]) # vocab_to_int 단어들의 숫자들을 갖고옴 
            else:
                sentence_ints.append(vocab_to_int["<UNK>"]) # vocab_to_int 에 없으면 unknown! 
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints) # sentence_int를 int에 추가한다 ! 
    return ints, word_count, unk_count

In [31]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count) # 여기서 이 결과는 각 단어들이 나온 모든 횟수를 다 더한것 !   
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))
# int_summaries는 단어들을 idx로 변환한 애들임 !  

Total number of words in headlines: 727325
Total number of UNKs in headlines: 11631
Percent of words that are UNK: 1.6%


In [33]:
int_summaries

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 [11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 [21, 22, 23, 24, 19, 25, 26, 27, 28, 29],
 [19, 30, 31, 8, 9, 10, 32, 33, 34, 35],
 [19, 30, 36, 8, 9, 10, 37, 35, 38, 39],
 [33, 40, 41, 42, 43, 44, 22897, 45, 46],
 [33, 0, 47, 48, 49, 50, 51, 52, 53],
 [54, 27, 55, 56, 19, 30, 57, 58, 59, 8, 60, 10],
 [19, 30, 61, 62, 63, 64, 65, 66, 67, 68],
 [19, 69, 70, 71, 72, 14, 73, 42, 74, 75, 10],
 [19, 30, 74, 71, 72, 14, 73, 76, 77, 78, 79],
 [33, 80, 81, 82, 83, 84, 85, 86, 87, 88],
 [19, 30, 89, 90, 9, 10, 33, 91, 22897, 92, 93],
 [94, 30, 95, 96, 97, 98, 99, 100, 22897, 101],
 [94, 30, 90, 102, 103, 15, 104, 0, 1, 105, 106, 107, 108, 109, 110],
 [33, 80, 81, 82, 83, 84, 85, 86, 87, 88],
 [111, 112, 113, 114, 115, 19, 30, 15, 116, 17, 8, 14, 9, 10],
 [76, 117, 118, 78, 119, 120, 19, 30, 35, 121],
 [122, 123, 124, 125, 126, 127],
 [19, 30, 128, 12, 129, 15, 116, 17],
 [33, 130, 131, 132, 59, 22897, 133, 134, 135],
 [136, 137, 138, 14],
 [40, 30, 128, 12, 129]

In [34]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [35]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe()) # 문장마다 단어 몇 개가 쓰였는지 !!! 

Summaries:
             counts
count  17184.000000
mean       9.155435
std        2.318872
min        1.000000
25%        8.000000
50%        9.000000
75%       11.000000
max       22.000000

Texts:
             counts
count  17184.000000
mean      34.170275
std        9.432231
min        2.000000
25%       28.000000
50%       34.000000
75%       41.000000
max       70.000000


In [51]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

46.0
49.0
54.0


In [52]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99)) # 몇개의 단어가 쓰였는지를 의미 !!! 

12.0
13.0
15.0


In [38]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [39]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 47
max_summary_length = 11
min_length = 2
unk_text_limit = 1
unk_summary_limit = 0

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

9020
9020


### Building the Model

In [40]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [41]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1]) #???
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [42]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [43]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [44]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, *_ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [45]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32)) 

    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [46]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
     # !
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size) # !
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input) #!
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [47]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [48]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [49]:
# Set the Hyperparameters
epochs = 100
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [50]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

C:\Users\CSH\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


AttributeError: module 'tensorflow.contrib.seq2seq' has no attribute 'DynamicAttentionWrapper'

In [64]:
f = open('word_counts.txt', 'wb')
pickle.dump(word_counts, f)
f.close()

In [44]:
f = open('vocab_to_int.txt', 'wb')
pickle.dump(vocab_to_int, f)
f.close()

In [45]:
f = open('int_to_vocab.txt', 'wb')
pickle.dump(int_to_vocab, f)
f.close()

In [46]:
f = open('int_summaries.txt', 'wb')
pickle.dump(int_summaries, f)
f.close()

In [47]:
f = open('int_texts.txt', 'wb')
pickle.dump(int_texts, f)
f.close()

## Training the Model

In [48]:
# Subset the data for training

start = 0
end = len(sorted_texts)

sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

The shortest text length: 3
The longest text length: 102


len(sorted_summaries)

In [ ]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1

update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "./best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

Epoch   1/100 Batch   20/166 - Loss:  6.430, Seconds: 4.32
Epoch   1/100 Batch   40/166 - Loss:  4.846, Seconds: 5.10
Average loss for this update: 5.425
New Record!
Epoch   1/100 Batch   60/166 - Loss:  4.678, Seconds: 5.07
Epoch   1/100 Batch   80/166 - Loss:  4.348, Seconds: 5.73
Epoch   1/100 Batch  100/166 - Loss:  4.271, Seconds: 6.13
Average loss for this update: 4.301
New Record!
Epoch   1/100 Batch  120/166 - Loss:  4.235, Seconds: 5.71
Epoch   1/100 Batch  140/166 - Loss:  4.296, Seconds: 6.88
Epoch   1/100 Batch  160/166 - Loss:  4.287, Seconds: 7.74
Average loss for this update: 4.278
New Record!
Epoch   2/100 Batch   20/166 - Loss:  4.067, Seconds: 4.50
Epoch   2/100 Batch   40/166 - Loss:  3.650, Seconds: 4.58
Average loss for this update: 3.811
New Record!
Epoch   2/100 Batch   60/166 - Loss:  3.586, Seconds: 5.30
Epoch   2/100 Batch   80/166 - Loss:  3.381, Seconds: 5.54
Epoch   2/100 Batch  100/166 - Loss:  3.368, Seconds: 6.05
Average loss for this update: 3.383
New R

Average loss for this update: 1.08
No Improvement.
Epoch  15/100 Batch   20/166 - Loss:  1.061, Seconds: 4.14
Epoch  15/100 Batch   40/166 - Loss:  0.973, Seconds: 4.76
Average loss for this update: 0.982
New Record!
Epoch  15/100 Batch   60/166 - Loss:  0.907, Seconds: 5.56
Epoch  15/100 Batch   80/166 - Loss:  0.907, Seconds: 5.47
Epoch  15/100 Batch  100/166 - Loss:  0.911, Seconds: 6.07
Average loss for this update: 0.938
New Record!
Epoch  15/100 Batch  120/166 - Loss:  1.060, Seconds: 5.78
Epoch  15/100 Batch  140/166 - Loss:  0.977, Seconds: 6.67
Epoch  15/100 Batch  160/166 - Loss:  1.030, Seconds: 7.89
Average loss for this update: 1.02
No Improvement.
Epoch  16/100 Batch   20/166 - Loss:  1.007, Seconds: 4.41
Epoch  16/100 Batch   40/166 - Loss:  0.889, Seconds: 4.94
Average loss for this update: 0.921
New Record!
Epoch  16/100 Batch   60/166 - Loss:  0.861, Seconds: 5.28
Epoch  16/100 Batch   80/166 - Loss:  0.873, Seconds: 5.36
Epoch  16/100 Batch  100/166 - Loss:  0.848, S

Average loss for this update: 0.549
No Improvement.
Epoch  28/100 Batch   60/166 - Loss:  0.517, Seconds: 5.04
Epoch  28/100 Batch   80/166 - Loss:  0.494, Seconds: 5.45
Epoch  28/100 Batch  100/166 - Loss:  0.508, Seconds: 5.72
Average loss for this update: 0.519
New Record!
Epoch  28/100 Batch  120/166 - Loss:  0.582, Seconds: 5.83
Epoch  28/100 Batch  140/166 - Loss:  0.523, Seconds: 6.73
Epoch  28/100 Batch  160/166 - Loss:  0.550, Seconds: 7.84
Average loss for this update: 0.548
No Improvement.
Epoch  29/100 Batch   20/166 - Loss:  0.551, Seconds: 4.15
Epoch  29/100 Batch   40/166 - Loss:  0.503, Seconds: 4.67
Average loss for this update: 0.512
New Record!
Epoch  29/100 Batch   60/166 - Loss:  0.474, Seconds: 5.32
Epoch  29/100 Batch   80/166 - Loss:  0.462, Seconds: 5.24
Epoch  29/100 Batch  100/166 - Loss:  0.481, Seconds: 6.12
Average loss for this update: 0.487
New Record!
Epoch  29/100 Batch  120/166 - Loss:  0.546, Seconds: 5.76
Epoch  29/100 Batch  140/166 - Loss:  0.502,

## Making Our Own Summaries

In [51]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    #text = clean_text(text)
    text = clean_texts[random]
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [52]:
random=19

In [53]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = text_to_seq(input_sentence)
#random = np.random.randint(0,len(clean_texts))
input_sentence = clean_texts[random]
text = text_to_seq(clean_texts[random]) # 우리 인풋에서 랜덤치를 뽑는거다 

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph("./"+checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(5,8)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text:  더불 전문 사모 집합 투자업 사모 펀드 업 진출 다양 금융 상품 출시 고객 기반 확대 계획 닭 예로 새벽 알리 울음소리 때문 어둠 걷어내 붉 태양 부르 광명 상징 인식 김 대표 정 유년 새해 불안 어둠 걷어내 우렁차 닭 울음 기울 차고 용기 있 활동 대한민국 모두 풍요 한해 되 기원 송 정훈

Text
  Word Ids:    [3760, 311, 8816, 9042, 3269, 8816, 6881, 7116, 13954, 10299, 5730, 12091, 8007, 7688, 12311, 15025, 13550, 2107, 1637, 15179, 6872, 15722, 827, 7792, 8088, 3241, 3185, 4024, 6669, 2986, 14920, 2204, 4967, 6895, 10681, 7084, 9849, 7792, 8088, 14852, 2107, 4569, 6161, 2065, 8715, 3614, 7417, 434, 4830, 6282, 7510, 10996, 9738, 5727, 15550]
  Input Words: 더불 전문 사모 집합 투자업 사모 펀드 업 진출 다양 금융 상품 출시 고객 기반 확대 계획 닭 예로 새벽 알리 울음소리 때문 어둠 걷어내 붉 태양 부르 광명 상징 인식 김 대표 정 유년 새해 불안 어둠 걷어내 우렁차 닭 울음 기울 차고 용기 있 활동 대한민국 모두 풍요 한해 되 기원 송 정훈

Summary
  Word Ids:       [13772, 3442, 5775, 9176, 2204, 4967]
  Response Words: 1 호 증권사 교보 김 대표


In [ ]:
clean_texts[13760]

In [56]:
randoms = [10,20,30,50,60,11100,2051,8695]

In [57]:

original_content = []
predict_title = []
original_title = []


checkpoint = "./best_model.ckpt"
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    for random in randoms: # 새로 추가된 부분 
        original_sum = clean_summaries[random]
        input_sentence = clean_texts[random]
        text = text_to_seq(clean_texts[random])

        #Multiply by batch_size to match the model's input parameters
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 

    # Remove the padding from the tweet
        pad = vocab_to_int["<PAD>"] 
        
        input_words = " ".join([int_to_vocab[i] for i in text])
        response_words = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        
        original_content.append(input_words)
        predict_title.append(response_words)
        original_title.append(original_sum)

        print('Original Text:', input_sentence)

        print('\nText')
        print('  Input Words: {}'.format(input_words))

        print('\nSummary')
        print('  Response Words: {}'.format(response_words))
        print('  Original Summary:', original_sum)
        print('\n\n\n')

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Text:  임은 영 삼성 증권 연구원 중국 시장 소형차 구매 세 감면 연장 자동차 금융 비중 확대 견 조 수요 성장 예상 현대 차 등 완성 차 업체 차량 전장화 트렌드 고려 현대 모비스 만도 올해 성장 지속 것 전망 유통 업종 올해 전망 밝

Text
  Input Words: 임은 영 삼성 증권 연구원 중국 시장 <UNK> 구매 세 감면 연장 자동차 금융 비중 확대 견 조 수요 성장 예상 현대 차 등 완성 차 업체 차량 <UNK> 트렌드 고려 현대 모비스 만도 올해 성장 지속 것 전망 유통 업종 올해 전망 밝

Summary
  Response Words: 현대 차 현대 상사 실적
  Original Summary:  2017 정유 업종 투자 기상도 반도체 맑 유통 림




Original Text:  나라 11 월 달러 부족 사태 통화 가치 48 절하 나이 지리 아 화 368 떨어지 최악 인플레이션 시름 앓 베네 수 엘 라 37 떨어지 중국 위 안화 65 하락 자본 유출 우려 증폭 유로화 33 내리 한국 원화 29 떨어지 아시아 국가 가운데 필리핀 페소 말레이시아 링 깃 화도 5 안팎 하락 일본 엔화 가치 1 년 사이 31 상승 시트 당시 달러 당 100 엔 선 밑돌 초강세 나타내 트럼프 당선 이후 빠르 약세 전환 모습 주식 시장 러시아 브라질 돋보이 러시아 지수 2009 년 이후 크 폭 인 522 뛰 브라질 보 베스 파 지수 389 오르 달러 기준 환산 691 의 상승률 주요 주가 지수 가운데 최고

Text
  Input Words: 나라 11 월 달러 부족 사태 통화 가치 48 절하 나이 지리 아 화 368 떨어지 최악 인플레이션 시름 앓 베네 수 엘 라 37 떨어지 중국 위 안화 65 하락 자본 유출 우려 증폭 유로화 33 내리 한국 원화 29 떨어지 아시아 국가 가운데 필리핀 페소 말레이시아 링 깃 화도 5 안팎 하락 일본

In [ ]:
original_content = pd.DataFrame(original_content, columns='original_content')
predict_title = pd.DataFrame(predict_title, columns='predict_title')
original_title = pd.DataFrame(original_title, columns='original_title')

result = pd.concat([original_content, predict_title, original_title], axis=1)
result